In [22]:
# ## graphql query to seach
# {
#   Get{
#     DocumentChunk(
#       nearText: {
#         concepts: ["musique"],
#         # distance: 0.9
#       }
#       groupBy: {  # How to group the results
#         path: ["local_path"]
#         groups: 10
#         objectsPerGroup: 10
#       }
#     ) {
#       _additional {
#         group {  # Data to be returned
#           id
#           groupedBy{ value }
#           count
#           hits {  # Actual properties to be retrieved
#             media_type
#             chunk
#             local_path
#             _additional {
#               id
#               distance
#             }
#           }
#         }
#       }
#     }
#   }
# }

In [28]:
import weaviate
client = weaviate.connect_to_local()
from langchain_text_splitters import CharacterTextSplitter
from weaviate.util import get_valid_uuid
from weaviate.classes.query import Filter, GeoCoordinate, MetadataQuery, QueryReference 
import weaviate.classes as wvc
from weaviate import WeaviateClient
from typing import List, Dict, Optional


In [34]:
response = client.collections.get("DocumentChunk").query.hybrid(
    query="gymnastique",
    limit=1000,
    filters=Filter.by_property("document_id").equal(get_valid_uuid(uuid="2a7a0d87-eaca-45f5-b0b0-db10b06ff8d8")),
    return_metadata=MetadataQuery(score=True),
)
response.objects

[Object(uuid=_WeaviateUUIDInt('53a1cd3c-bdde-4c3f-b486-d7c736d9ed89'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=0.699999988079071, explain_score=None, is_consistent=None, rerank_score=None), properties={'original_public_path': 'original file path', 'document_id': UUID('2a7a0d87-eaca-45f5-b0b0-db10b06ff8d8'), 'media_type': 'text', 'start_offset': 12931.0, 'local_path': './Cellule_biologie.txt', 'chunk': "Structures spécifiques aux procaryotes\n\nProcessus cellulaires\n\nMétabolisme et cycle cellulaire\n\nEntre deux divisions successives du cycle cellulaire, les cellules se développent grâce à leur métabolisme. Le métabolisme cellulaire est le processus par lequel chaque cellule exploite les nutriments qu'elle absorbe afin de se maintenir en vie et de se reproduire. Le métabolisme se divise en deux grandes parties\xa0: d'une part le catabolisme, par lequel les cellules dégradent les molécules complexes en espèces chimiques pl

In [37]:
from typing import Literal
from pydantic import BaseModel
from enum import Enum
import weaviate.classes as wvc

class DocumentChunk(BaseModel):
    chunk: str
    document_id: str
    local_path: str
    original_public_path: str
    media_type: str
    start_offset: int
    end_offset: int
    



#///////////
questions = client.collections.get("DocumentChunk")
response = questions.query.hybrid(
    query="danse",
    # distance=0.95,
    return_metadata=wvc.query.MetadataQuery(distance=True, score=True),
    group_by=wvc.query.GroupBy(
        prop="local_path",
        number_of_groups=10,
        objects_per_group=10
    )
)

for local_path, group in response.groups.items():  # View by group
    max_distance = group.max_distance
    min_distance = group.min_distance
    print(f"Group {local_path} has {len(group.objects)} objects")
    # print(group)
    for _documentChunk in group.objects:
        documentChunk = DocumentChunk.model_validate({**_documentChunk.properties, "document_id": str(_documentChunk.properties["document_id"])})
        print(documentChunk)
        print(_documentChunk.metadata.)



Group ./Gymnastique_rythmique.txt has 10 objects
chunk="Chaque enchaînement est noté de la manière suivante\xa0: la note finale est donnée par la somme des trois notes (exécution, artistique et difficulté) après soustraction des éventuelles pénalités. Le total de points atteignable peut varier selon les pays et les catégories, en fonction des divers ajustements.\n\n\nDifficultés\n\nSelon le code de pointage 2017-2020, la note de difficulté est composée de quatre parties\xa0: les difficultés corporelles, les combinaisons de pas de danse, les éléments dynamiques avec rotation et lancer (aussi appelés risques) et les difficultés d'engin." document_id='602d2cb0-dafb-4eb9-b437-be5f0f3a3bcf' local_path='./Gymnastique_rythmique.txt' original_public_path='original file path' media_type='text' start_offset=4696 end_offset=5271
GroupByMetadataReturn(distance=0.0)
chunk="Sauts\n\nLa gymnaste effectue une figure en l'air en sautant. Il existe plusieurs sortes de sauts\xa0: enjambé, 1\xa0ou 2\xa0bo

DocumentChunk(chunk="On peut noter également que la chanteuse serbe Nevena Božović, a aussi participée en 2007 à l'édition junior du concours.\n\n\nCartes postales\n\nLes cartes postales, sur le thème «\xa0Danser Israël\xa0», sont censées mettre en valeur la beauté d'Israël. Dans celles-ci, les participants vont ainsi danser dans différents lieux du pays. Les styles de danses varient du hip-hop au ballet en passant par la danse contemporaine. Chacune d'entre elles est tournée dans un lieu différent, incluant Masada, Césarée, Eilat ou encore le Mont Hermon. Accompagnant les participants, de nombreux danseurs et compagnies de danseurs professionnels israéliens y sont également présents, tels le Ballet Israélien, la Batsheva Dance Company, Anna Aronov, Or Kahlon ou encore Rona-Lee Shim'on,.\n\n\nRépétitions", document_id='369c7fff-3187-43e8-9c64-c2d5700d1a77', local_path='./Concours_Eurovision_de_la_chanson_2019.txt', original_public_path='original file path', media_type='text', start_off

In [20]:
x = {
  "data": {
    "Get": {
      "DocumentChunk": [
        {
          "_additional": {
            "group": {
              "count": 10,
              "groupedBy": {
                "value": "./Concours_Eurovision_de_la_chanson_2019.txt"
              },
              "hits": [
                {
                  "_additional": {
                    "distance": 0.68932486,
                    "id": "83f37945-4403-494f-a1b9-be6c5c7d46dc"
                  },
                  "chunk": "Résultats finaux\n\nCette édition est remportée, avec 498 points, par les Pays-Bas, représentés par Duncan Laurence et sa chanson Arcade. Il s'agit de la cinquième victoire du pays, la précédente étant en 1975. Les Pays-Bas ne remportent ni le vote des jurys, ni le télévote ; la Macédoine du Nord ayant remporté le premier avec 247 points et la Norvège le second avec 291 points. En deuxième position vient l'Italie avec un total de 472 points. Le podium est complété par la Russie avec 370 points. Viennent ensuite la Suisse avec 364 points, qui obtient ainsi son meilleur résultat depuis 1993, et la Suède avec 334 points. Le Top 10 est complété par la Norvège, la Macédoine du Nord — qui obtient son meilleur résultat —, l'Azerbaïdjan, l'Australie et l'Islande.",
                  "local_path": "./Concours_Eurovision_de_la_chanson_2019.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.71474874,
                    "id": "ce48ba30-320d-455d-bfdb-ab1cd444dd49"
                  },
                  "chunk": "Vous lisez un « bon article » labellisé en 2019.\n\n\nDare to Dream\n\nLisbonne 2018 Rotterdam 2020 (annulé)Rotterdam 2021\n\nmodifier \n\nLe Concours Eurovision de la chanson 2019 est la 64e édition du concours. Il a lieu à Tel Aviv, en Israël, à la suite de la victoire de Netta lors de l'édition 2018, avec la chanson Toy. C'est la troisième fois que le pays accueille l'Eurovision, après les éditions 1979 et 1999. Les demi-finales ont lieu les 14 et 16 mai 2019, et la finale se déroule le 18 mai 2019. Quarante-et-un pays participent à cette édition dont le slogan est Dare to Dream (en français Osez rêver).\n\n\nLes Pays-Bas remportent cette édition avec la chanson Arcade interprétée par Duncan Laurence, avec un total de 498 points. C'est la cinquième victoire du pays à l'Eurovision, la dernière étant en 1975. L'Italie arrive en deuxième position avec 472 points. La Russie termine troisième avec 370 points. La Suisse et la Suède complètent le Top 5.",
                  "local_path": "./Concours_Eurovision_de_la_chanson_2019.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.7156385,
                    "id": "d54cdd44-868e-4214-8112-397e9103d814"
                  },
                  "chunk": "Parmi les pays du Big Five, seule l'Italie atteint le Top 10, arrivant 2e. La France arrive en 16e place. Les autres qualifiés d'office terminent en fin de classement : l'Espagne arrive 22e ; Israël, pays hôte, 23e ; l'Allemagne termine 25e. La dernière place échoit au Royaume-Uni avec seulement 11 points.\n\n\nConférence de presse du gagnant\n\nUne conférence de presse a lieu après la finale. Duncan Laurence s'exprime sur sa victoire, notamment son ressenti au concours, sa chanson ainsi que sur la trace qu'il laissera dans l'histoire de l'Eurovision en tant que vainqueur. Marquant la clôture du concours, c'est lors de cette conférence de presse que Jon Ola Sand, superviseur exécutif, remet à la délégation néerlandaise un premier cahier des charges pour l'accueil de l'édition suivante.\n\n\nTableaux des votes\n\nPremière demi-finale\n\nDeuxième demi-finale\n\nFinale\n\nIncidents et controverses\n\nDemandes religieuses",
                  "local_path": "./Concours_Eurovision_de_la_chanson_2019.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.7208304,
                    "id": "f971bc14-608c-4530-a090-6908d9c36b24"
                  },
                  "chunk": "Lors de la distribution des votes des jurys, aucun gagnant clair n'émerge dans un premier temps. Le vote est mené alternativement par l'Italie et la Russie, qui maintiennent une courte avance face aux autres pays, notamment les Pays-Bas, la Suède, l'Azerbaïdjan et la Macédoine du Nord. Cependant, les douze points consécutifs autrichien et britannique — 12e et 13e dans l'ordre du vote — attribués à la Macédoine du Nord permettent à cette dernière de prendre la tête du classement. L'écart se creuse alors progressivement, la Macédoine du Nord prenant une légère avance, alors que les pays suivants voient leur classement varier. Finalement, lors du dernier vote annoncé — le vote israélien —, la Suède prend sur le fil la tête du vote des jurys d'une très courte avance. La procédure télévisée se conclut donc avec la Suède en première position avec 239 points, la Macédoine du Nord deuxième avec 237 points et les Pays-Bas troisièmes avec 231 points.",
                  "local_path": "./Concours_Eurovision_de_la_chanson_2019.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.726684,
                    "id": "23f83e03-8676-4493-a582-080e296bfe63"
                  },
                  "chunk": "Autres prix\n\nPrix Marcel-Bezençon\n\nLes prix Marcel-Bezençon sont remis pour la première fois durant le Concours Eurovision de la chanson 2002 à Tallinn en Estonie pour honorer les meilleures chansons en compétition lors de la finale. Fondés par Christer Björkman (représentant de la Suède lors de l'Eurovision 1992 et producteur des concours 2013 et 2016) et Richard Herrey (membre des Herreys et vainqueur suédois du concours 1984), les prix portent le nom du créateur de la compétition annuelle, Marcel Bezençon. Remis tous les ans, ils sont répartis en trois catégories : le prix de la presse attribué par les médias accrédités à la meilleure chanson, le prix de la meilleure performance artistique attribué au meilleur artiste par les commentateurs du concours et, enfin, le prix de la meilleure composition attribué par les auteurs-compositeurs participants aux meilleurs compositeurs de la soirée. En 2019, les candidats récompensés sont :\n\n\nOGAE",
                  "local_path": "./Concours_Eurovision_de_la_chanson_2019.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.7268905,
                    "id": "a4ee53ed-5079-4b79-b23d-e0d7d6644d7a"
                  },
                  "chunk": "Enfin, en dernier entracte, Madonna, grande invitée de cette édition 2019, se produit et interpète sa chanson Like a Prayer et son nouveau titre Future avec le rappeur Quavo. Son apparition à l'Eurovision aura coûté 1,3 MUSD, somme payée par le philanthrope israélo-canadien Sylvan Adams. Arrivée en Israël le mardi 14 mai malgré une incertitude sur sa participation à la finale, l'artiste américaine s'est entourée d'une équipe de 135 personnes dont 40 choristes et 25 danseurs.\n\n\nVote télévisé\n\nLa présentation des résultats subit cette année un léger changement. Une fois les résultats des jurys nationaux annoncés par les porte-paroles des pays participants, le télévote est annoncé non plus dans l'ordre ascendant des points mais en suivant le classement des jurys. Ainsi, l'annonce des résultats du télévote débute par le pays classé dernier par les jurys et se termine par le pays classé premier par les jurys,.",
                  "local_path": "./Concours_Eurovision_de_la_chanson_2019.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.73042583,
                    "id": "cb7b9789-dee7-4152-b6c7-389beeafa11e"
                  },
                  "chunk": "OGAE\n\nL'Organisation générale des amateurs de l'Eurovision (OGAE) est l'un des deux plus grands fan clubs de l'Eurovision. Elle possède des sections dans toute l'Europe et a été créée en 1984 en Finlande. Tous les pays prenant part ou qui ont pris part à la compétition annuelle peuvent avoir leur propre section OGAE. Les autres pays du monde sont, quant à eux, regroupés dans l'« OGAE Reste du monde », créée en 2004. Chaque année, les clubs ont l'occasion de voter, suivant le modèle de l'Eurovision, pour leurs favoris.\n\n\nLes résultats finaux du vote sont annoncés le 30 avril 2019. L'Italie y termine en tête avec 411 points, suivie de la Suisse avec 406 points et des Pays-Bas avec 401 points. Le tableau ci-dessous montre les dix premiers à l'issue du vote.\n\n\nPrix Barbara-Dex",
                  "local_path": "./Concours_Eurovision_de_la_chanson_2019.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.731497,
                    "id": "ef6b919d-4684-45d2-bc18-aa6f3919418c"
                  },
                  "chunk": "Selon les nouveaux résultats, les douze points du « jury » biélorusse sont attribués à Malte. Les Pays-Bas remportent finalement le concours avec 498 points, l'Italie en totalise 472 et la Russie en obtient 370. Les classements finaux de nombreux pays sont influencés. Par ailleurs, la Macédoine du Nord remporte finalement le vote du jury avec 247 points alors que la Norvège termine en réalité à la 18e place du classement des jurys.",
                  "local_path": "./Concours_Eurovision_de_la_chanson_2019.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.73318696,
                    "id": "97701aad-95fa-46d0-a2da-4700a6c44700"
                  },
                  "chunk": "Similairement, la jurée suédoise Lina Hedlund semble avoir également voté à l'envers, lors de la deuxième demi-finale. En effet, lors de celle-ci, Lina Hedlund place les Pays-Bas et la Suisse respectivement dernier et avant-dernier lors de la demi-finale, avant de les classer premier et deuxième lors de la finale. De plus, en demi-finale, Lina Hedlund classe ainsi l'Autriche comme étant sa chanson préférée, permettant à l'Autriche de recevoir 8 points du jury suédois. Ni Lina Hedlund, ni le diffuseur suédois SVT, ni l'UER n'ont commenté,.",
                  "local_path": "./Concours_Eurovision_de_la_chanson_2019.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.74607813,
                    "id": "c60dd779-5850-4c21-8505-6b9838cc2bd5"
                  },
                  "chunk": "Manifestations politiques\n\nL'organisation de l'Eurovision par Israël engendre de nombreuses protestations en raison de l'occupation de la Palestine par Israël. Deux manifestations politiques notables ont lieu pendant la finale. Tout d'abord, entre ses prestations de Like a Prayer et Future, Madonna interprète un monologue pendant lequel elle fait allusion à «  en nous » et déclare « ils pensent que nous ne sommes pas conscients de leurs crimes. Nous le savons, mais nous ne sommes juste pas prêts à agir », ce qui est interprété comme une référence au conflit. Pendant la prestation de Future, deux danseurs portant respectivement les drapeaux israélien et palestinien sur le dos de leurs costumes sont filmés mains jointes en signe d'unité pendant que Quavo chantait les vers « Not everyone is coming to the future/Not everyone is learning from the past. » (en français « Tout le monde ne viendra pas dans le futur/Tout le monde n'apprend pas du passé. »).",
                  "local_path": "./Concours_Eurovision_de_la_chanson_2019.txt",
                  "media_type": "text"
                }
              ],
              "id": 0
            }
          }
        },
        {
          "_additional": {
            "group": {
              "count": 10,
              "groupedBy": {
                "value": "./Cellule_biologie.txt"
              },
              "hits": [
                {
                  "_additional": {
                    "distance": 0.8139063,
                    "id": "28e1055f-6748-4593-87e5-14b00dd470d9"
                  },
                  "chunk": "Cet article concerne l'unité de base du vivant. Pour la subdivision d'aile d'insectes, voir Cellule (entomologie). Pour les autres significations, voir Cellule.\n\n\nLa cellule — du latin cellula « chambre de moine » — est l'unité biologique structurelle et fonctionnelle fondamentale de tous les êtres vivants connus. C'est la plus petite unité vivante capable de se reproduire de façon autonome. La science qui étudie les cellules est appelée biologie cellulaire.",
                  "local_path": "./Cellule_biologie.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.82119155,
                    "id": "8bed51c3-1922-4945-97aa-698716856749"
                  },
                  "chunk": "Méthodes d'étude de la cellule in vitro\n\nMicroscopie\n\nLa microscopie optique ou photonique (résolution de +/- 0,25µm en lumière visible) permet l'observation de la structure des cellules eucaryotes. En effet, les microscopes photoniques peuvent grossir environ 1000 fois la taille du spécimen. Mais ce type de microscope n'est pas assez puissant pour l'étude des organites de la cellule.",
                  "local_path": "./Cellule_biologie.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.82465893,
                    "id": "6bbbbbab-f539-4394-b5b1-8908ad276cfe"
                  },
                  "chunk": "Lors de la transcription, des ARN polymérases produisent un brin d'ARN complémentaire au brin d'ADN codant. L'information génétique est portée par la séquence nucléotidique de l'ADN, reproduite sur l'ARN messager lors de la transcription. Cette séquence est ensuite lue par des ribosomes afin de polymériser les acides aminés dans l'ordre spécifié par la succession de groupes de trois nucléotides sur l'ARN messager, chacun de ces triplets, appelés codons, correspondant à un acide aminé donné ; c'est cette correspondance entre codons et acides aminés qu'on appelle code génétique.\n\n\nLocomotion ou motilité\n\nLes unicellulaires peuvent se déplacer afin de trouver de la nourriture ou d'échapper à leurs prédateurs. Flagelles et cils sont les principaux moyens de motilité cellulaire.",
                  "local_path": "./Cellule_biologie.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.8303254,
                    "id": "896a7ede-79bb-410e-b0aa-bb665911cd67"
                  },
                  "chunk": "Paroi cellulaire\n\nDe nombreux types de cellules de procaryotes et d'eucaryotes possèdent une paroi cellulaire. Celle-ci protège la cellule des actions chimiques et mécaniques de son environnement et ajoute une couche protectrice supplémentaire par-dessus la membrane plasmique. Les différents types de cellules tendent à produire des parois de nature chimique différente : la paroi pectocellulosique des plantes est constituée essentiellement de cellulose, la paroi des mycètes est constituée essentiellement de chitine, et la paroi bactérienne est constituée essentiellement de peptidoglycane.\n\n\nStructures spécifiques aux procaryotes\n\nProcessus cellulaires\n\nMétabolisme et cycle cellulaire",
                  "local_path": "./Cellule_biologie.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.8315941,
                    "id": "0cb9fef2-b72b-4e2c-a2af-ce7beae8bda3"
                  },
                  "chunk": "Origine de la multicellularité\n\nLa multicellularité a émergé à partir d'organismes unicellulaires un grand nombre de fois au cours de l'évolution et ne s'observe pas uniquement chez les eucaryotes : certains procaryotes tels que des cyanobactéries, des myxobactéries, actinomycètes, Magnetoglobus multicellularis ou encore des archées du genre Methanosarcina, présentent des organisations multicellulaires. Cependant, ce sont bien chez les eucaryotes que des organismes multicellulaires sont apparus, et ce parmi six groupes : les animaux, les mycètes, les algues brunes, les algues rouges, les algues vertes et les plantes. La multicellularité peut être apparue à partir de colonies d'organismes interdépendants, voire à partir d'organismes en symbiose.",
                  "local_path": "./Cellule_biologie.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.83236146,
                    "id": "72c6d99b-88cb-4ae0-8248-b300a52811e9"
                  },
                  "chunk": "Chez les organismes multicellulaires, les cellules peuvent se déplacer par exemple lors de la cicatrisation des plaies, lors de la réponse immunitaire, ou encore lors de la formation de métastases tumorales. Ainsi, les leucocytes (globules blancs) se déplacent jusqu'à la plaie afin d'y tuer les micro-organismes susceptibles d'y provoquer des infections. La motilité cellulaire fait intervenir de nombreux récepteurs, des mécanismes de réticulation, d'assemblage, de liaison ou encore d'adhérence de protéines, ainsi que des protéines motrices, parmi d'autres types de protéines. Le processus se déroule en trois temps : saillie de la pointe avant de la cellule, adhérence de l'avant de la cellule et « désadhérence » du reste de la surface cellulaire, et contraction du cytosquelette pour tirer la cellule vers l'avant. Chacune de ces étapes est gérée par des forces produites par des segments particuliers du cytosquelette. Comme pour la prolifération, la question de savoir si la motilité des cellules d'un multicellulaire est spontanée, comme pour les unicellulaires, ou doit faire l'objet de stimulations fait l'objet d'un débat.",
                  "local_path": "./Cellule_biologie.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.8409277,
                    "id": "0c11faf4-3432-422f-b544-b16ce1aef374"
                  },
                  "chunk": "Dans le cas des organismes unicellulaires, il est généralement accepté que les cellules prolifèrent spontanément, sans avoir besoin de stimulation. Dans le cas des organismes multicellulaires, cette question fait l'objet d'un débat. De nombreux auteurs défendent l'idée que ces cellules requièrent une stimulation pour proliférer, d'autres au contraire considèrent que la quiescence est le résultat de contraintes agissant sur les cellules quiescentes,. Pour la modélisation du comportent des cellules, les deux points de vue sont couramment utilisés.\n\n\nBiosynthèse des protéines",
                  "local_path": "./Cellule_biologie.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.8481598,
                    "id": "49b62dce-56ea-4a58-9da4-d7e3c50196f7"
                  },
                  "chunk": "Les plus anciennes traces de multicellularité ont été identifiées chez des organismes apparentés aux cyanobactéries qui vivaient il y a entre 3 et 3,5 milliards d'années. Parmi d'autres fossiles d'organismes multicellulaires, on compte également Grypania spiralis,, dont la nature biologique exacte demeure cependant débattue, ainsi que les fossiles des schistes paléoprotérozoïques du groupe fossile de Franceville, au Gabon.\n\n\nL'évolution d'organismes multicellulaires à partir d'ancêtres unicellulaires a été reproduite au laboratoire à travers des expériences d'évolution expérimentale utilisant la prédation comme vecteur de pression de sélection.\n\n\nOrigine des cellules\n\nL'origine des cellules est intimement liée à l'origine de la vie, aux origines de l'histoire évolutive du vivant.\n\n\nOrigines des premières cellules",
                  "local_path": "./Cellule_biologie.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.84889185,
                    "id": "b108a9af-3131-4c90-9a7e-a0655d1a5062"
                  },
                  "chunk": "Pluricellularité\n\nUn organisme multicellulaire est constitué de plusieurs cellules, par opposition à un organisme unicellulaire.\n\n\nSpécialisation cellulaire",
                  "local_path": "./Cellule_biologie.txt",
                  "media_type": "text"
                },
                {
                  "_additional": {
                    "distance": 0.8501,
                    "id": "a1d34e48-67b8-450b-a3e9-616cc13326ca"
                  },
                  "chunk": "Il existe différents types d'organites dans une cellule. Certains sont généralement uniques, comme l'appareil de Golgi, tandis que d'autres sont présents en de très nombreux exemplaires — des centaines, voire des milliers — comme les mitochondries, les chloroplastes, les peroxysomes et les lysosomes. Le cytosol est le fluide gélatineux qui entoure les organites dans le cytoplasme.\n\n\nOrganites présents chez tous les êtres vivants\n\nOrganites des cellules d'eucaryotes\n\n\nÉléments du système endomembranaire :\n\n\nStructures extracellulaires\n\nDe nombreuses cellules possèdent également des structures entièrement ou partiellement situées à l'extérieur de la membrane plasmique. Ces structures ne sont donc pas protégées de l'environnement de la cellule par une membrane semiperméable. L'assemblage de ces structures implique que leurs constituants soient transportés hors de la cellule par des processus spécifiques.\n\n\nParoi cellulaire",
                  "local_path": "./Cellule_biologie.txt",
                  "media_type": "text"
                }
              ],
              "id": 1
            }
          }
        }
      ]
    }
  }
}

In [21]:

from typing import List, Dict
from pydantic import BaseModel, Field
from collections import defaultdict

# Define Pydantic models for output
class Chunk(BaseModel):
    chunk: str
    # local_path: str
    media_type: str
    score: float


class GroupedDocument(BaseModel):
    local_path: str
    chunks: List[Chunk] = Field(default_factory=list)
    max_score: float = float('-inf')


data = x['data']['Get']['DocumentChunk']

acc = defaultdict(lambda: GroupedDocument(local_path=''))

for doc in data:
    local_path = doc['_additional']['group']['groupedBy']['value']
    
    chunks = [Chunk(
        chunk=hit['chunk'],
        # local_path=hit['local_path'],
        media_type=hit['media_type'],
        score=hit['_additional']['distance']
    ) for hit in doc['_additional']['group']['hits']]
    
    max_score = max(chunk.score for chunk in chunks)
    acc[local_path].local_path = local_path
    acc[local_path].chunks.extend(chunks)
    acc[local_path].max_score = max(acc[local_path].max_score, max_score)

# Convert defaultdict to regular list of dicts
result = [group.dict() for group in acc.values()]

result


[{'local_path': './Concours_Eurovision_de_la_chanson_2019.txt',
  'chunks': [{'chunk': "Résultats finaux\n\nCette édition est remportée, avec 498\xa0points, par les Pays-Bas, représentés par Duncan Laurence et sa chanson Arcade. Il s'agit de la cinquième victoire du pays, la précédente étant en 1975. Les Pays-Bas ne remportent ni le vote des jurys, ni le télévote\xa0; la Macédoine du Nord ayant remporté le premier avec 247\xa0points et la Norvège le second avec 291\xa0points. En deuxième position vient l'Italie avec un total de 472\xa0points. Le podium est complété par la Russie avec 370\xa0points. Viennent ensuite la Suisse avec 364\xa0points, qui obtient ainsi son meilleur résultat depuis 1993, et la Suède avec 334\xa0points. Le Top 10 est complété par la Norvège, la Macédoine du Nord —\xa0qui obtient son meilleur résultat\xa0—, l'Azerbaïdjan, l'Australie et l'Islande.",
    'media_type': 'text',
    'score': 0.68932486},
   {'chunk': "Vous lisez un «\xa0bon article\xa0» labellisé en